# Load to database

Rename and organize hazard, network and customer data.

Pre-requisites:
- python packages (`jupyter`, `psycopg2`, `fiona`).
- PostgreSQL database with PostGIS extension activated.
- database connection details saved to `config.json`

In [ ]:
import glob
import json
import os

import psycopg2
import fiona
import geopandas

from shapely import wkt
from shapely.geometry import shape

Load config from `config.json` (copy template from `config-template.json` and edit details).

In [ ]:
with open('config.json', 'r') as fh:
    config = json.load(fh)

Connect to the database.

In [ ]:
conn = psycopg2.connect(**config['database'])

## Create tables

Set up database tables for data loading:
- hazard outlines and scenarios
- infrastructure network nodes and edges
- boundaries and regions
- buildings

`hazard_modelled` table will contain modelled hazard extents

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS hazard_modelled")
        cur.execute("""CREATE TABLE 
        hazard_modelled (
            id serial PRIMARY KEY,
            return_period int,
            source_id int,
            oia_id varchar(20),
            catchment_id int,
            catchment_name varchar(50),
            frms_id varchar(20),
            geom geometry(GEOMETRY,27700)
        );
        """)

`hazard_historic` table will contain historic hazard extents

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS hazard_historic")
        cur.execute("""CREATE TABLE
        hazard_historic (
            id serial PRIMARY KEY,
            event_group varchar(20),
            event_code int,
            source_id int,
            oia_id varchar(50),
            name varchar(50),
            start_date date,
            end_date date,
            geom geometry(GEOMETRY,27700)
        );
        """)

`infrastructure_nodes` table will contain nodes from all infrastructure sectors

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS infrastructure_nodes CASCADE")
        cur.execute("""CREATE TABLE
        infrastructure_nodes (
            oia_id varchar(21) PRIMARY KEY,
            source_id varchar(50),
            name varchar(60),
            sector varchar(22),
            asset_type varchar(100),
            centroid geometry(POINT,27700), -- single centroid
            catchment geometry(POLYGON,27700), -- service area/catchment
            geometry geometry(GEOMETRY,27700), -- outline or point as provided
            protected boolean,
            customers double precision,
            floor_area_m2 double precision,
            floor_level varchar(12),
            elec_id varchar(24) REFERENCES infrastructure_nodes (oia_id)
        );
        """)

`infrastructure_edges` table will contain physical edges from infrastructure sectors where available (e.g. road, rail)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS infrastructure_edges")
        cur.execute("""CREATE TABLE
        infrastructure_edges (
            oia_id varchar(40) PRIMARY KEY,
            source_id varchar(30),
            name varchar(60),
            from_oia_id varchar(20) REFERENCES infrastructure_nodes (oia_id),
            to_oia_id varchar(20) REFERENCES infrastructure_nodes (oia_id),
            sector varchar(20),
            geometry geometry(LINESTRING,27700), -- line as provided
            protected boolean,
            customers double precision
        );
        """)

`interdependency_edges` table will contain non-physical or inferred edges of the infrastructure network (e.g. electricity distribution)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS interdependency_edges")
        cur.execute("""CREATE TABLE
        interdependency_edges (
            oia_id varchar(40) PRIMARY KEY,
            from_oia_id varchar(20) REFERENCES infrastructure_nodes (oia_id),
            to_oia_id varchar(20) REFERENCES infrastructure_nodes (oia_id),
            from_sector varchar(20),
            to_sector varchar(20),
            geometry geometry(LINESTRING,27700), -- line as synthesised
            protected boolean
        );
        """)

`zones` table will contain administrative zones if/when available (e.g. national boundaries, Flood Risk Management Systems)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS zones")
        cur.execute("""CREATE TABLE
        zones (
            id serial PRIMARY KEY,
            original_id varchar(30),
            name varchar(40),
            zone_type varchar(20),
            geom geometry(GEOMETRY,27700)
        );
        """)

## Load data

Load data as provided, transforming/renaming as necessary
- modelled floods
- historic floods
- airports
- electricity cables, overhead lines, substations, towers
- gas sites, pipes
- ports
- railway lines, stations
- road links, nodes
- wastewater treatment sites
- water treatment works
- residential and non-residential buildings

In [ ]:
data_path = config['files']['data_path']
data_path

## Nodes
### Airports
- 1x multi-polygon per airport.
- Customer numbers based on CAA 2017 total passenger counts (divided by 365).

In [ ]:
airports = geopandas.read_file(
    os.path.join(data_path, 'airport', 'airports.shp')
).drop(
    columns=['ID', 'CLASSIFICA', 'FEATCODE', 'Name2', 'Name3', 'this_perio', 
             'rpt_apt__1', 'rpt_apt_na', 'terminal', 'transit', 'total2017']
).rename(
    columns={
      'LTIS_ID': 'source_id',
      'Name1': 'name',
      'AADF': 'customers'
    }
).reset_index()
airports['geometry'] = airports.geometry.apply(lambda geom: wkt.loads(geom.to_wkt()))
airports['sector'] = 'airport'
airports['asset_type'] = 'airport'
airports['centroid'] = airports.geometry.centroid
airports['oia_id'] = airports['index'].apply(lambda n: "airp_{}".format(n))
airports.drop(columns=['index'], inplace=True)
airports['protected'] = False
airports['elec_id'] = None

In [ ]:
#airports.geometry[0].wkt

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_nodes 
            WHERE sector = 'airport'
            """)
        for airport in airports.itertuples():
            cur.execute("""INSERT INTO infrastructure_nodes 
            (
                source_id,
                name,
                customers,
                geometry,
                sector,
                asset_type,
                centroid,
                oia_id,
                protected,
                elec_id
            )
            VALUES
            (
                %s,
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                %s
            )
            """, (
                airport.source_id,
                airport.name,
                airport.customers,
                airport.geometry.wkb_hex, 27700,
                airport.sector,
                airport.asset_type,
                airport.centroid.wkb_hex, 27700,
                airport.oia_id,
                airport.protected,
                airport.elec_id
            ))

### Ports
- 1x multi-polygon per port.
- Port usage data is based the total freight movement (combined: domestic, international, incoming, outgoing)
- Port usage data is appended from an additional .csv file

In [ ]:
ports = geopandas.read_file(
    os.path.join(data_path, 'port', 'ports.shp')
).drop(
    columns=['ID', 'SITETHEME','FEATCODE','Port', ]
).rename(
    columns={
      'LTIS_ID': 'source_id',
      'DISTNAME': 'name',
      'CLASSIFICA': 'asset_type'  
    }
).reset_index()
ports['geometry'] = ports.geometry.apply(lambda geom: wkt.loads(geom.to_wkt()))
ports['sector'] = 'port'
ports['centroid'] = ports.geometry.centroid
ports['oia_id'] = ports['index'].apply(lambda n: "port_{}".format(n))
ports.drop(columns=['index'], inplace=True)
ports['protected'] = False
ports['elec_id'] = None
ports['customers'] = None

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_nodes 
            WHERE sector = 'port'
            """)
        for port in ports.itertuples():
            cur.execute("""INSERT INTO infrastructure_nodes 
            (
                source_id,
                name,
                customers,
                geometry,
                sector,
                asset_type,
                centroid,
                oia_id,
                protected,
                elec_id
            )
            VALUES
            (
                %s,
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                %s
            )
            """, (
                port.source_id,
                port.name,
                port.customers,
                port.geometry.wkb_hex, 27700,
                port.sector,
                port.asset_type,
                port.centroid.wkb_hex, 27700,
                port.oia_id,
                port.protected,
                port.elec_id
            ))

### Gas
- 1x point per gas node.
- Customer data is the based on the LDZ and distributed evenly across the nodes in the. North Wales being an exception where no gas node exists, and is instead assumed dependent on the nearest node at ??

In [ ]:
gas = geopandas.read_file(
    os.path.join(data_path, 'gasnetwork', 'sites_1.shp')
).drop(
    columns=['NOTES1', 'SITE_TYPE','STATUS','ACTION_DTT','OPER_AND_S', 'x_cent_bng','y_cent_bng']
).rename(
    columns={
      'GDO_GID': 'source_id',
      'SITE_NAME': 'name',
      'LDZ': 'asset_type',
      'gas_demand': 'customers'  
    }
).reset_index()
gas['geometry'] = gas.geometry.apply(lambda geom: wkt.loads(geom.to_wkt()))
gas['sector'] = 'gas'
gas['centroid'] = gas.geometry.centroid
gas['oia_id'] = gas['index'].apply(lambda n: "gasx_{}".format(n))
gas.drop(columns=['index'], inplace=True)
gas['protected'] = False
gas['elec_id'] = None
gas

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_nodes 
            WHERE sector = 'gas'
            """)
        for gasnode in gas.itertuples():
            cur.execute("""INSERT INTO infrastructure_nodes 
            (
                source_id,
                name,
                customers,
                geometry,
                sector,
                asset_type,
                centroid,
                oia_id,
                protected,
                elec_id
            )
            VALUES
            (
                %s,
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                %s
            )
            """, (
                gasnode.source_id,
                gasnode.name,
                gasnode.customers,
                gasnode.geometry.wkb_hex, 27700,
                gasnode.sector,
                gasnode.asset_type,
                gasnode.centroid.wkb_hex, 27700,
                gasnode.oia_id,
                gasnode.protected,
                gasnode.elec_id
            ))

### Electricity
- 1x multi-polygon per substation.
- Customer data is to be based on NRD dataset such that floor area is a proxy for demand
- Ratio for energy use across Residential:Commercial:Industry is   ?:?:? 

In [ ]:
electricity = geopandas.read_file(
    os.path.join(data_path, 'electricity', 'substations.shp')
).drop(
    columns=['x_cent_BNG','y_cent_BNG']
).rename(
    columns={
      'Country': 'source_id',
      'NG_NAME': 'name',
    }
).reset_index()
electricity['geometry'] = electricity.geometry.apply(lambda geom: wkt.loads(geom.to_wkt()))
electricity['sector'] = 'electricity'
electricity['asset_type'] = 'substation'
electricity['centroid'] = electricity.geometry.centroid
electricity['oia_id'] = electricity['index'].apply(lambda n: "elec_{}".format(n))
electricity.drop(columns=['index'], inplace=True)
electricity['protected'] = False
electricity['elec_id'] = None
electricity['customers'] = None

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_nodes 
            WHERE sector = 'electricity'
            """)
        for substation in electricity.itertuples():
            cur.execute("""INSERT INTO infrastructure_nodes 
            (
                source_id,
                name,
                customers,
                geometry,
                sector,
                asset_type,
                centroid,
                oia_id,
                protected,
                elec_id
            )
            VALUES
            (
                %s,
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                %s
            )
            """, (
                substation.source_id,
                substation.name,
                substation.customers,
                substation.geometry.wkb_hex, 27700,
                substation.sector,
                substation.asset_type,
                substation.centroid.wkb_hex, 27700,
                substation.oia_id,
                substation.protected,
                substation.elec_id
            ))

### Water Supply
- 1x multi-polygon per WTP.
- Customer data is to be based on NRD dataset such that floor area is a proxy for demand
- Ratio for water use across Residential:Commercial:Industry is   ?:?:? 

In [ ]:
water = geopandas.read_file(
    os.path.join(data_path, 'watersupply', 'osm_wtp.shp')
).drop(
    columns=['addr_house','addr_postc','capacity','landuse','man_made', 'operator','Shape_Leng','Shape_Area','x_bng','y_bng',]
).rename(
    columns={
      'id': 'source_id',
    }
).reset_index()
water['geometry'] = water.geometry.apply(lambda geom: wkt.loads(geom.to_wkt()))
water['sector'] = 'water_supply'
water['asset_type'] = 'treatment_plant'
water['centroid'] = water.geometry.centroid
water['oia_id'] = water['index'].apply(lambda n: "wsup_{}".format(n))
water.drop(columns=['index'], inplace=True)
water['protected'] = False
water['elec_id'] = None
water['customers'] = None

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_nodes 
            WHERE sector = 'water_supply'
            """)
        for treatment_plant in water.itertuples():
            cur.execute("""INSERT INTO infrastructure_nodes 
            (
                source_id,
                name,
                customers,
                geometry,
                sector,
                asset_type,
                centroid,
                oia_id,
                protected,
                elec_id
            )
            VALUES
            (
                %s,
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                %s
            )
            """, (
                treatment_plant.source_id,
                treatment_plant.name,
                treatment_plant.customers,
                treatment_plant.geometry.wkb_hex, 27700,
                treatment_plant.sector,
                treatment_plant.asset_type,
                treatment_plant.centroid.wkb_hex, 27700,
                treatment_plant.oia_id,
                treatment_plant.protected,
                treatment_plant.elec_id
            ))

### Wastewater
- 1x node per treatment plant.
- Customer data is calculated based on the capacity of the plant (units are persons.equivalent) 

In [ ]:
wastewater = geopandas.read_file(
    os.path.join(data_path, 'wastewater', 'WW_Points.shp')
).drop(
    columns=[]
).rename(
    columns={
      'uwwCode': 'source_id',
      'uwwName': 'name',  
      'uwwCapacit': 'customers'  
    }
).reset_index()
wastewater['geometry'] = wastewater.geometry.apply(lambda geom: wkt.loads(geom.to_wkt()))
wastewater['sector'] = 'wastewater'
wastewater['asset_type'] = 'treatment_plant'
wastewater['centroid'] = wastewater.geometry.centroid
wastewater['oia_id'] = wastewater['index'].apply(lambda n: "wwat_{}".format(n))
wastewater.drop(columns=['index'], inplace=True)
wastewater['protected'] = False
wastewater['elec_id'] = None

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_nodes 
            WHERE sector = 'wastewater'
            """)
        for treatment_plant in wastewater.itertuples():
            cur.execute("""INSERT INTO infrastructure_nodes 
            (
                source_id,
                name,
                customers,
                geometry,
                sector,
                asset_type,
                centroid,
                oia_id,
                protected,
                elec_id
            )
            VALUES
            (
                %s,
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                %s
            )
            """, (
                treatment_plant.source_id,
                treatment_plant.name,
                treatment_plant.customers,
                treatment_plant.geometry.wkb_hex, 27700,
                treatment_plant.sector,
                treatment_plant.asset_type,
                treatment_plant.centroid.wkb_hex, 27700,
                treatment_plant.oia_id,
                treatment_plant.protected,
                treatment_plant.elec_id
            ))

## Edges

### Roads
- 1x motor vehicle allocation estimation (AADT) per road segment.
- Join table 'AADF-data-major-roads.csv' based on 'CP_Number' and 'CP_Number' fields.
- CP Numbers have been assumed based on the CP of the nearest road segment from the official dataset. The road network being imported here is from OS.
- Customer disruptions will be based on vehicle.km (all motor vehicles count * length_km)

In [ ]:
roads = geopandas.read_file(
    os.path.join(data_path, 'road', 'AADT_Ard_Mrd.shp')
).drop(
    columns=['formOfWay','loop','function']
).rename(
    columns={
      'roadNumber': 'source_id',
      'name1': 'name',
    }
).reset_index()
roads['geometry'] = roads.geometry.apply(lambda geom: wkt.loads(geom.to_wkt()))
roads['sector'] = 'road'
roads['oia_id'] = roads['index'].apply(lambda n: "road_{}".format(n))
roads.drop(columns=['index'], inplace=True)
roads['from_oia_id'] = None
roads['to_oia_id']= None
roads['protected'] = False
roads['customers'] = None
roads

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_edges 
            WHERE sector = 'road'
            """)
        for road in roads.itertuples():
            cur.execute("""INSERT INTO infrastructure_edges
            (
                oia_id,
                source_id,
                name,
                from_oia_id,
                to_oia_id,
                sector,
                geometry,
                protected,
                customers
            )
            VALUES
            (
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s
            )
            """, (
                road.oia_id,
                road.source_id,
                road.name,
                road.from_oia_id,
                road.to_oia_id,
                road.sector,
                road.geometry.wkb_hex, 27700,
                road.protected,
                road.customers
            ))

### National Receptor Dataset

24 million residential, 2 million non-residential properties.

Population (`AvgPop2016`) is recorded as `customers` for residential properties. Population averaging/matching was done by CH2M.

Floor area and floor level are as provided.

Land use type (`LTIS_Desc`) is recorded in `asset_type` for residential and non-residential properties.

In [ ]:
nrd_file = os.path.join(data_path, 'nrd_gdb', 'LTIS_NRD.gdb')

In [ ]:
fiona.listlayers(nrd_file)

In [ ]:
def insert_nrd_node(f, conn):
    props = f['properties']
    geom = shape(f['geometry'])
    with conn:
        with conn.cursor() as cur:
            cur.execute("""INSERT INTO infrastructure_nodes 
            (
                oia_id,
                source_id,
                geometry,
                centroid,
                sector,
                asset_type,
                customers,
                floor_area_m2,
                floor_level
            )
            VALUES
            (
                %s,
                %s,
                ST_SetSRID(%s::geometry, %s),
                ST_SetSRID(%s::geometry, %s),
                %s,
                %s,
                %s,
                %s,
                %s
            )
            """, (
                props['oia_id'],
                props['source_id'],                
                geom.wkb_hex, 27700,        
                geom.wkb_hex, 27700,                
                props['sector'],
                props['asset_type'], # asset_type
                props['customers'], # customers
                props['floor_area_m2'], # floor_area_m2
                props['floor_level'] # floor_level
            ))

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("""DELETE FROM infrastructure_nodes 
            WHERE sector in ('NRD_RES', 'NRD_NONRES')
            """)

In [ ]:
with fiona.open(nrd_file, layer='NRD2014_LTIS_FloorArea_RES') as source:
    for i, f in enumerate(source):
        if (i+1) % 1000 == 0:
            print(i)
        p = f['properties']
        props = {
            'oia_id': 'nrdR_{}'.format(f['id']),
            'sector': 'NRD_RES',
            'source_id': p['LOCALID'],
            'asset_type': p['LTIS_Desc'],
            'customers': p['AvgPop2016'],
            'floor_area_m2': p['FLOORAREA'],
            'floor_level': p['FLOORLEVEL']
        }
        f['properties'] = props
        insert_nrd_node(f, conn)

In [ ]:
with fiona.open(nrd_file, layer='NRD2014_LTIS_FloorArea_NONRES') as source:
    for i, f in enumerate(source):
        if (i+1) % 1000 == 0:
            print(i)
        p = f['properties']
        props = {
            'oia_id': 'nrdN_{}'.format(f['id']),
            'sector': 'NRD_NONRES',
            'source_id': p['UPRN'],
            'asset_type': p['LTIS_Desc'],
            'customers': None,
            'floor_area_m2': p['FLOORAREA'],
            'floor_level': p['FLOORLEVEL']
        }
        f['properties'] = props
        insert_nrd_node(f, conn)

## Hazards

### Historic

In [ ]:
def insert_hazard_historic(hazard, conn, group):    
    with conn:
        with conn.cursor() as cur:
            for outline in hazard.itertuples():
                cur.execute("""INSERT INTO hazard_historic 
                (
                    event_group,
                    event_code,
                    source_id,
                    oia_id,
                    name,
                    start_date,
                    end_date,
                    geom
                )
                VALUES
                (
                    %s,
                    %s,
                    %s,
                    %s,
                    %s,
                    %s,
                    %s,
                    ST_SetSRID(%s::geometry, %s)
                )
                """, (
                    group,
                    outline.event_code,
                    outline.source_id,
                    outline.oia_id,
                    outline.name,
                    outline.start_date,
                    outline.end_date,
                    outline.geometry.wkb_hex, 27700
                ))

In [ ]:
for group in ['01062007_to_31072007', '05122015_to_01072016', '23112013_to_05032014']:
    hazard = geopandas.read_file(
        os.path.join(data_path, 'Flood', 'RecordedFloodOutlines_Final', 
                     'RecordedFloodOutlines_{}.shp'.format(group))
    ).drop(
        columns=['BNDRY_SRC', 'FLOOD_SRC', 'FLOOD_CAUS', 'FLUVIAL_IN', 'TIDAL_IND', 'COASTAL_IN',
                'HFM_IND', 'SHAPE_Leng', 'Shape_Le_1', 'Shape_Area']
    ).rename(
        columns={
            'EVENT_CODE': 'event_code',
            'OUTLINE_CO': 'source_id',
            'NAME': 'name',
            'START_DATE': 'start_date',
            'END_DATE': 'end_date'
        }
    ).reset_index()
    hazard['oia_id'] = hazard['index'].apply(lambda n: "{}_{}".format(group, n))
    insert_hazard_historic(hazard, conn, group)

### Probabilistic

Modelled floods for 30, 100, 1000 year returns

In [ ]:
hazard_T30 = geopandas.read_file(
    os.path.join(data_path, 'Flood', 'RoFRS', 'RoFRS_1in30yr.shp')
).drop(
    columns=['Shape_Leng', 'Shape_Area']
).rename(
    columns={
        'Catch_Na': 'catchment_name',
        'Catch_No': 'catchment_id',
        'GROUP_CODE': 'frms_id'
    }
).reset_index().rename(
    columns={'index':'source_id'}
)
hazard_T30['oia_id'] = hazard_T30.source_id.apply(lambda n: "T30_{}".format(n))
hazard_T30

In [ ]:
hazard_T100 = geopandas.read_file(
    os.path.join(data_path, 'Flood', 'RoFRS', 'RoFRS_1in100yr.shp')
).drop(
    columns=['Shape_Leng', 'Shape_Area']
).rename(
    columns={
        'Catch_Na': 'catchment_name',
        'Catch_No': 'catchment_id',
        'GROUP_CODE': 'frms_id'
    }
).reset_index().rename(
    columns={'index':'source_id'}
)
hazard_T100['oia_id'] = hazard_T100.source_id.apply(lambda n: "T100_{}".format(n))
hazard_T100

In [ ]:
hazard_T1000 = geopandas.read_file(
    os.path.join(data_path, 'Flood', 'RoFRS', 'RoFRS_1in1000yr.shp')
).drop(
    columns=['Shape_Leng', 'Shape_Area']
).rename(
    columns={
        'Catch_Na': 'catchment_name',
        'Catch_No': 'catchment_id',
        'GROUP_CODE': 'frms_id'
    }
).reset_index().rename(
    columns={'index':'source_id'}
)
hazard_T1000['oia_id'] = hazard_T1000.source_id.apply(lambda n: "T1000_{}".format(n))
hazard_T1000

In [ ]:
def insert_hazard(hazard, conn, rp):    
    with conn:
        with conn.cursor() as cur:
            for outline in hazard.itertuples():
                cur.execute("""INSERT INTO hazard_modelled 
                (
                    return_period,
                    source_id,
                    oia_id,
                    catchment_id,
                    catchment_name,
                    frms_id,
                    geom
                )
                VALUES
                (
                    %s,
                    %s,
                    %s,
                    %s,
                    %s,
                    %s,
                    ST_SetSRID(%s::geometry, %s)
                )
                """, (
                    rp,
                    outline.source_id,
                    outline.oia_id,
                    outline.catchment_id,
                    outline.catchment_name,
                    outline.frms_id,
                    outline.geometry.wkb_hex, 27700
                ))

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("TRUNCATE hazard_modelled;")

In [ ]:
insert_hazard(hazard_T30, conn, 30)
insert_hazard(hazard_T100, conn, 100)
insert_hazard(hazard_T1000, conn, 1000)